In [ ]:
%pip install nltk
%pip install tweepy
%pip install pandas
%pip install wordcloud
%pip install textblob
%pip install seaborn

In [ ]:
import tweepy
import json
import csv
import sys
import time
import re
import warnings
import string
import pandas as pd
import ast
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import Stream
from wordcloud import WordCloud
from textblob import TextBlob
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

In [77]:
ckey = 'o7qidRODRWm5u07PtAWrCpSqf'
csecret = '1Yn60OYMJl5kRCSM0XQ4Lskxh6hjM4VMz51OdsS6dM9DQuQ4MV'
atoken = '1315792402234441730-Dt7AydgVGYI5VN9RRQfnR8Endh0WAc'
asecret = 'jhfiDfB53Z27EqHfvfKbNdo0w2SWLqyyoAkILqgSoaGMK'

OAUTH_KEYS = {'consumer_key':ckey, 'consumer_secret':csecret, 'access_token_key':atoken, 'access_token_secret':asecret}
auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])

api = tweepy.API(auth)

In [78]:
def TweetTemizle(Dataset):
    yedek=[]
    print ("Tweetler Temizleniyor...")

    pattern = re.compile('http[s]?://(?:[A-Za-z0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|#[A-Za-z0-9]+|$[A-Za-z0-9]+|@[A-Za-z0-9_]+|[,.^_$*%-;鶯!?’"“”:=|…]+|\n')
    for i in range(len(Dataset["Tweet_Değiştirilmiş"])):        
        Dataset["Tweet_Değiştirilmiş"][i]=pattern.sub('', Dataset["Tweet_Değiştirilmiş"][i])
        
    print ("Tweetler Temizlendi!")
    return Dataset

In [79]:
def toDataFrame(tweets):
    print ("Veriler Kayıt Ediliyor...")
    # COnvert to data frame    
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
    DataSet = pd.DataFrame()


    DataSet['Tweet_Id'] = [tweet.id for tweet in tweets]
    DataSet['Tweet'] = [tweet.text for tweet in tweets]
    DataSet['Tweet_Değiştirilmiş'] = [tweet.text for tweet in tweets]
    DataSet['Tweet_Tarih'] = [tweet.created_at for tweet in tweets]
    DataSet['Kullanıcı_Id'] = [tweet.user.id for tweet in tweets]  
    DataSet['Kullanıcı_Adı'] = [tweet.user.name for tweet in tweets]       
    DataSet['Dil'] = [tweet.lang for tweet in tweets]
    
    """"
    DataSet['userCreateDt'] = [tweet.user.created_at for tweet in tweets]
    DataSet['tweetRetweetCt'] = [tweet.retweet_count for tweet in tweets]
    DataSet['tweetFavoriteCt'] = [tweet.favorite_count for tweet in tweets]
    DataSet['tweetSource'] = [tweet.source for tweet in tweets]
    DataSet['userScreen'] = [tweet.user.screen_name for tweet in tweets]
    DataSet['userDesc'] = [tweet.user.description for tweet in tweets]
    DataSet['userFollowerCt'] = [tweet.user.followers_count for tweet in tweets]
    DataSet['userFriendsCt'] = [tweet.user.friends_count for tweet in tweets]
    DataSet['userLocation'] = [tweet.user.location for tweet in tweets]
    DataSet['userTimezone'] = [tweet.user.time_zone for tweet in tweets]
    DataSet['Coordinates'] = [tweet.coordinates for tweet in tweets]
    DataSet['GeoEnabled'] = [tweet.user.geo_enabled for tweet in tweets] 
    users_retweeted = []    
    for tweet in tweets:
        if tweet.place:
            tweets_place.append(tweet.place.full_name)
        else:
            tweets_place.append('null')
    
    DataSet['TweetPlace'] = [i for i in tweets_place]
    DataSet['UserWhoRetweeted'] = [i for i in users_retweeted]
    """
    print ("Veriler Kayıt Edildi!")
    return DataSet

In [80]:
def KelimeAyristirma(DataSet):
    data_test = DataSet.copy()
    stop = stopwords.words('turkish')
    data_test["Tweet_Değiştirilmiş"] = data_test["Tweet_Değiştirilmiş"].str.lower().str.split()
    data_test["Tweet_Değiştirilmiş"] = data_test["Tweet_Değiştirilmiş"].apply(lambda x: [item for item in x if item not in stop])
    data_test.to_csv('deneme.csv',index=False)
    print(data_test.tail())

def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity 

In [81]:
if (not api):
    print ("Bağlantı Yapılamadı!")
    sys.exit(-1)
else:
    print ("Bağlantı Yapıldı!")
    
    aranan=input("Aranacak Kelimeyi Girin: ")
    
    print ("Kelimeye Uygun Tweetler Sorgulanıyor...")
    
    cursor = tweepy.Cursor(api.search_tweets, q=aranan, result_type="new", geocode="55.0000,4.0000,1000km",lang='en',count=100)
    
    results=[]
    for item in cursor.items(1000): # Remove the limit to 1000
            results.append(item)
            
    print ("Kelimeye Uygun Tweetler Sorgulandı!")
    
    DataSet = toDataFrame(results)    
    TweetTemizle(DataSet)
    DataSet.to_csv('tw.csv',index=False)

Bağlantı Yapıldı!
Kelimeye Uygun Tweetler Sorgulanıyor...
Kelimeye Uygun Tweetler Sorgulandı!
Veriler Kayıt Ediliyor...
Veriler Kayıt Edildi!
Tweetler Temizleniyor...
Tweetler Temizlendi!


In [82]:
Veri=pd.read_csv('tw.csv')

Veri.head()

,Tweet_Id,Tweet,Tweet_Değiştirilmiş,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil
0,1597313762058735616,@elonmusk Me: Who writes the most embarrassing tweet today?\nElon Musk: Hold my beer! https://t.co/5ntGcSEVyn,Me Who writes the most embarrassing tweet todayElon Musk Hold my beer,2022-11-28 19:37:32+00:00,158650645,Rudolf Gottfried 🥝,en
1,1597313500971700224,Wish Elon Musk would just fuck off.,Wish Elon Musk would just fuck off,2022-11-28 19:36:29+00:00,206400467,Nick,en
2,1597313497268125697,@elonmusk Elon Musk is getting salty cos someone treats him like he just treated many 😂. Maybe it's time to learn t… https://t.co/UO7P5BOK1F,Elon Musk is getting salty cos someone treats him like he just treated many 😂 Maybe its time to learn t,2022-11-28 19:36:29+00:00,455847450,Sebastian 🇺🇦,en
3,1597313430117306368,"@mustafaergisi elon musk's real problem isn't apple's 30% commission, it's that apple no longer wants to buy advertising on twitter",elon musks real problem isnt apples commission its that apple no longer wants to buy advertising on twitter,2022-11-28 19:36:13+00:00,4901020462,frederic kavita,en
4,1597313400081907712,@DeanObeidallah @elonmusk Elon Musk must be deported. To OR Tambo airport asap,Elon Musk must be deported To OR Tambo airport asap,2022-11-28 19:36:05+00:00,1453858919726657546,Manish Punish,en


In [83]:
Veri.tail()

,Tweet_Id,Tweet,Tweet_Değiştirilmiş,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil
995,1596998602391719936,Elon Musk is the best thing to happen to Twitter and the motor industry.\n\nI am being treated well and not being tortured.,Elon Musk is the best thing to happen to Twitter and the motor industryI am being treated well and not being tortured,2022-11-27 22:45:12+00:00,1289756592670494721,WikiJewSoc🕎🌹,en
996,1596998174975352832,"@BurnsideNotTosh Elon Musk must feel conflicted,when he's been so vocal supporting the freedom of speech for everyo… https://t.co/SjxOhj7kAH",Elon Musk must feel conflictedwhen hes been so vocal supporting the freedom of speech for everyo,2022-11-27 22:43:30+00:00,552765332,Andre Crabtree III,en
997,1596998071162114049,"@madrid_mike Nope, it’s always been exactly the same as it is now. Unless your suffering from a Dysphoria about Elo… https://t.co/tXnAordI45",Nope its always been exactly the same as it is now Unless your suffering from a Dysphoria about Elo,2022-11-27 22:43:05+00:00,921956874349694976,Alan West,en
998,1596997466590949376,Bill Clinton Will FREAK When He See Who Elon Musk Just Allowed Back on T... https://t.co/Wh8JJzzviI via @YouTube,Bill Clinton Will FREAK When He See Who Elon Musk Just Allowed Back on T via,2022-11-27 22:40:41+00:00,1493974191292485646,Peter Holand,en
999,1596996685036945408,@POTUS You have alot to learn from Elon musk then.,You have alot to learn from Elon musk then,2022-11-27 22:37:35+00:00,1682904026,Francóise.🇺🇬,en


In [84]:
KelimeAyristirma(Veri)


                Tweet_Id  \
995  1596998602391719936   
996  1596998174975352832   
997  1596998071162114049   
998  1596997466590949376   
999  1596996685036945408   

                                                                                                                                            Tweet  \
995                   Elon Musk is the best thing to happen to Twitter and the motor industry.\n\nI am being treated well and not being tortured.   
996  @BurnsideNotTosh Elon Musk must feel conflicted,when he's been so vocal supporting the freedom of speech for everyo… https://t.co/SjxOhj7kAH   
997  @madrid_mike Nope, it’s always been exactly the same as it is now. Unless your suffering from a Dysphoria about Elo… https://t.co/tXnAordI45   
998                              Bill Clinton Will FREAK When He See Who Elon Musk Just Allowed Back on T... https://t.co/Wh8JJzzviI via @YouTube   
999                                                                                   

In [85]:
savedTweets = Veri.copy()
savedTweets['Subjectivity'] = savedTweets['Tweet_Değiştirilmiş'].apply(getSubjectivity)
savedTweets['Polarity'] = savedTweets['Tweet_Değiştirilmiş'].apply(getPolarity)
savedTweets.to_csv('duygu.csv')
savedTweets.head()

,Tweet_Id,Tweet,Tweet_Değiştirilmiş,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil,Subjectivity,Polarity
0,1597313762058735616,@elonmusk Me: Who writes the most embarrassing tweet today?\nElon Musk: Hold my beer! https://t.co/5ntGcSEVyn,Me Who writes the most embarrassing tweet todayElon Musk Hold my beer,2022-11-28 19:37:32+00:00,158650645,Rudolf Gottfried 🥝,en,0.5,0.5
1,1597313500971700224,Wish Elon Musk would just fuck off.,Wish Elon Musk would just fuck off,2022-11-28 19:36:29+00:00,206400467,Nick,en,0.6,-0.4
2,1597313497268125697,@elonmusk Elon Musk is getting salty cos someone treats him like he just treated many 😂. Maybe it's time to learn t… https://t.co/UO7P5BOK1F,Elon Musk is getting salty cos someone treats him like he just treated many 😂 Maybe its time to learn t,2022-11-28 19:36:29+00:00,455847450,Sebastian 🇺🇦,en,0.5,0.5
3,1597313430117306368,"@mustafaergisi elon musk's real problem isn't apple's 30% commission, it's that apple no longer wants to buy advertising on twitter",elon musks real problem isnt apples commission its that apple no longer wants to buy advertising on twitter,2022-11-28 19:36:13+00:00,4901020462,frederic kavita,en,0.2,0.2
4,1597313400081907712,@DeanObeidallah @elonmusk Elon Musk must be deported. To OR Tambo airport asap,Elon Musk must be deported To OR Tambo airport asap,2022-11-28 19:36:05+00:00,1453858919726657546,Manish Punish,en,0.0,0.0


In [86]:
savedTweets.tail()

,Tweet_Id,Tweet,Tweet_Değiştirilmiş,Tweet_Tarih,Kullanıcı_Id,Kullanıcı_Adı,Dil,Subjectivity,Polarity
995,1596998602391719936,Elon Musk is the best thing to happen to Twitter and the motor industry.\n\nI am being treated well and not being tortured.,Elon Musk is the best thing to happen to Twitter and the motor industryI am being treated well and not being tortured,2022-11-27 22:45:12+00:00,1289756592670494721,WikiJewSoc🕎🌹,en,0.3000,1.000
996,1596998174975352832,"@BurnsideNotTosh Elon Musk must feel conflicted,when he's been so vocal supporting the freedom of speech for everyo… https://t.co/SjxOhj7kAH",Elon Musk must feel conflictedwhen hes been so vocal supporting the freedom of speech for everyo,2022-11-27 22:43:30+00:00,552765332,Andre Crabtree III,en,0.2500,0.250
997,1596998071162114049,"@madrid_mike Nope, it’s always been exactly the same as it is now. Unless your suffering from a Dysphoria about Elo… https://t.co/tXnAordI45",Nope its always been exactly the same as it is now Unless your suffering from a Dysphoria about Elo,2022-11-27 22:43:05+00:00,921956874349694976,Alan West,en,0.1875,0.125
998,1596997466590949376,Bill Clinton Will FREAK When He See Who Elon Musk Just Allowed Back on T... https://t.co/Wh8JJzzviI via @YouTube,Bill Clinton Will FREAK When He See Who Elon Musk Just Allowed Back on T via,2022-11-27 22:40:41+00:00,1493974191292485646,Peter Holand,en,0.0000,0.000
999,1596996685036945408,@POTUS You have alot to learn from Elon musk then.,You have alot to learn from Elon musk then,2022-11-27 22:37:35+00:00,1682904026,Francóise.🇺🇬,en,0.0000,0.000
